In [78]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [79]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [80]:
with open("mydata.json", "rt") as INFILE:
    data=json.load(INFILE)

In [81]:
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
        'areaType=nation',
        'areaName=England'
    ]
    deaths = {
       "date":"date",
        "newDailyNsoDeathsByDeathDate":"newDailyNsoDeathsByDeathDate",
        "newDeaths28DaysByDeathDateChange":"newDeaths28DaysByDeathDateChange",
        "cumPeopleVaccinatedCompleteByVaccinationDate":"cumPeopleVaccinatedCompleteByVaccinationDate"
    }
    api = Cov19API(filters=filters, structure=deaths)
    mydata=api.get_json()
   # print(mydata)
    #data.to_pickle("mydata.pkl")
    #convert(mydata)
    return mydata # return data read from the API

def convert(whatever):
    with open("mydata.json", "wt") as OUTF:
        json.dump(whatever, OUTF)

In [82]:
def wrangle_data(rawdata):
    datalist=data['data']

    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()

    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")


    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])


    index=pd.date_range(startdate, enddate, freq='D')
    patdata=pd.DataFrame(index=index, columns=['newDailyNsoDeathsByDeathDate', 'newDeaths28DaysByDeathDateChange', 'cumPeopleVaccinatedCompleteByVaccinationDate'])

    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['newDailyNsoDeathsByDeathDate', 'newDeaths28DaysByDeathDateChange', 'cumPeopleVaccinatedCompleteByVaccinationDate']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(patdata.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                patdata.loc[date, column]=value
            
    patdata.fillna(0.0, inplace=True)
    return patdata


# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
patdata=wrangle_data(data) # df is the dataframe for plotting

In [83]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    apidata=access_api()
    global df
    df=wrangle_data(apidata)
    refresh_graph()
    apibutton.icon="check"

    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='info',
    tooltip="Keep calm and carry on",
    icon='download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

Button(button_style='info', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Keep ca…

In [84]:
def plot(data):
    patdata=data
    patdata.plot( logy=True)
    patdata.plot( logx=True, logy=True, kind='scatter', y=['newDailyNsoDeathsByDeathDate'], x=['cumPeopleVaccinatedCompleteByVaccinationDate'])

    agecols=wdg.SelectMultiple(
        options=['newDailyNsoDeathsByDeathDate', 'newDeaths28DaysByDeathDateChange', 'cumPeopleVaccinatedCompleteByVaccinationDate'], # options available
        value=['cumPeopleVaccinatedCompleteByVaccinationDate'], # initial value
        rows=3, # rows of the selection box
        description='Test stats:',
        disabled=False,
        layout={'width': 'max-content'}
)

def patdata_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        patdata.plot(logy=True,  y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
        plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(patdata_graph, {'graphcolumns': agecols})

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=agecols.value
    if current==agecols.options[2]:
        other=agecols.options[1]
    elif current==agecols.options[1]:
        other=agecols.options[0]
    else:
        other=agecols.options[2]
    agecols.value=other # forces the redraw
    agecols.value=current # now we can change it back



graph=wdg.interactive_output(plot, {'daata': agecols})

display(agecols, output)

patdata.to_pickle("patdata.pkl")

SelectMultiple(description='Test stats:', index=(2,), layout=Layout(width='max-content'), options=('newDailyNs…

Output()